## PDF Parsing

In [1]:
import sys
sys.path.append('../src/')
import ReadPDF as r

In [2]:
# Extract by Section
pdf_path = "4444_East.pdf"
sections = r.process_into_sections(pdf_path)
sections_text = r.select_Section(sections, 2)
one_section = '\n'.join(sections[1].text)

In [3]:
# Extract by Subsection
subsection_dict = r.process_file(pdf_path)
one_subsection = r.select_Subsection(subsection_dict, 2, 1)
print(one_subsection)

The subject property at 4444 East 26th Street, Vernon, California is located on the southwestern intersection of East 26th Street and Ayers Avenue.  The subject property was inspected by Joseph Kim of Partner on October 28, 2021.  The weather at the time of the site visit was sunny and in the mid-70s (degrees Fahrenheit). According to the Los Angeles County Assessor, the subject property is legally described as OM 3-19-27 EX OF R/W AND STS LOT 3 DIV 105 REG 48 and is owned by LBA RVI – Company VIII, LLC. Please refer to Figure 1: Site Location Map, Figure 2: Site Plan, Figure 3: Topographic Map, and Appendix A: Site Photographs for the location and site characteristics of the subject property.


## LLM Prompting

In [4]:
from langchain_community.llms import Ollama
llm = Ollama(model="phi")

In [5]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [6]:
%%time
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""Based only on the context below, retrieve the following data field given in input.
Forget previous questions.
If you cannot find the answer, say N/A.
If it takes more than 100 characters to respond, say "Timed out".
Do not take more than 10 seconds to respond.

<context>
{context}
</context>

What is {input}?""")

document_chain = create_stuff_documents_chain(llm, prompt)

CPU times: total: 812 ms
Wall time: 867 ms


In [7]:
%%time
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Name of Assessor/Appraisal District Agency",
    "context": [Document(page_content=one_subsection)]
})

CPU times: total: 46.9 ms
Wall time: 2.63 s


' Los Angeles County Assessor\n\n'

In [ ]:
%%time
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Property Legal Description",
    "context": [Document(page_content=one_subsection)]
})

In [ ]:
%%time
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Property Owner Name",
    "context": [Document(page_content=one_subsection)]
})

In [ ]:
%%time
from langchain_core.documents import Document

document_chain.invoke({
    "input": "When did the owner acquire the property?",
    "context": [Document(page_content=one_subsection)]
})

In [ ]:
%%time
from langchain_core.documents import Document

document_chain.invoke({
    "input": "What was the source of the property owner name and acquisition date?",
    "context": [Document(page_content=one_subsection)]
})